In [1]:


import sys
sys.path.insert(0,'./pyMOE-dev')
# sys.path.insert(0,'../../../7. Tools/pyMOE-dev')
# sys.path.insert(0,'./pyMOE-dev')

from matplotlib import pyplot as plt 
import numpy as np 

from scipy.constants import micro, nano, milli
from scipy import ndimage

import pyMOE as moe 

In [2]:

def alternate_transitions_symmetric(x, transitions, start_value=1):
    assert np.all(np.diff(transitions) > 0), "transitions should be monotonic increasing"
    y = np.full_like(x, start_value)

    # x_mod = np.mod(x, )
    for i in range(len(transitions)-1):
        mask = np.logical_and(abs(x) >= transitions[i], abs(x) < transitions[i+1])
        y[mask] = start_value * (-1) ** (i+1)
    mask = abs(x) >= transitions[-1]  # Add this line to handle the last transition
    y[mask] = start_value * (-1) ** len(transitions)
    return y


def minimum_step(vector):

    return np.min(np.diff(vector))


def dammann_grating_periodic(x,transitions, period=1, start_value=1):
    # assert np.all(np.array(transitions)<=0.5), "transitions should be <= 0.5"

    transitions = np.array(transitions)*period

    if len(transitions) >1:
        print("Minimum step: %e"%(minimum_step(transitions)))
    # x = clip_remainder(x, period)
    half_period = period/2
    # x = np.mod(np.abs(x+half_period), period)-half_period
    x = np.mod(x, period)

    y = alternate_transitions_symmetric(x,transitions=transitions, start_value=start_value)
    return y



def dammann_2d(XX,YY,transitions_x=None, period_x=1, transitions_y=None, period_y=1, start_value_x=1, start_value_y=1):
    x = XX[0,:]
    y = YY[:,0]

    if transitions_x is not None:
        grating_x = dammann_grating_periodic(x, transitions_x, period=period_x, start_value=start_value_x)
    else:
        grating_x = np.ones_like(x)
    
    if transitions_y is not None:
        grating_y = dammann_grating_periodic(y, transitions_y, period=period_y, start_value=start_value_y)
    else:  
        grating_y = np.ones_like(y)

    grating = np.outer(grating_y, grating_x)
    return grating




In [3]:
transitions_1x11 = [0.06857, 0.20885, 0.44467, 0.52930, 0.72101, 0.72854, 0.86437]
transitions_1x13 = [0.17765, 0.31352, 0.41244, 0.49846, 0.57633, 0.70857, 0.73041]


# Damman mask klayout directly


In [ ]:
import pya 
def create_poly_dicing_corner(length, width):
    poly = pya.DPolygon([ 
    pya.DPoint(0, 0), pya.DPoint(0, length),  pya.DPoint(width, length),
    pya.DPoint(width, width),  pya.DPoint(length, width), pya.DPoint(length, width),
    pya.DPoint(length, 0)
    ])
    return poly

def create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer="layer127"):
    cell_corner_single = layout.create_cell('corner_instance')

    cell_corner = layout.create_cell('corners')
    layer_corners = layout.layer(layer)
    corner_polygon = create_poly_dicing_corner(corner_length, corner_width)

    cell_corner_single.shapes(layer_corners).insert(corner_polygon)


    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R0, pya.DPoint(-field_width/2, -field_height/2))
                                )
    cell_corner.insert(instance)

    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R270, pya.DPoint(-field_width/2, field_height/2))
                                )
    cell_corner.insert(instance)


    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R180, pya.DPoint(field_width/2, field_height/2))
                                )
    cell_corner.insert(instance)

    instance = pya.DCellInstArray(cell_corner_single.cell_index(),
                                pya.DTrans(pya.DTrans.R90, pya.DPoint(field_width/2, -field_height/2))
                                )
    cell_corner.insert(instance)


    cell_corner.flatten(-1, True)

    return cell_corner



def create_label_cell(layout, text, position=(0,0), rotate=True, mag=1000, layer="layer127"):
    

    cell_label = layout.create_cell('label')
    layer_label = layout.layer(layer)


    gen = pya.TextGenerator.default_generator()
    
    region = gen.text(text, layout.dbu, mag)
    if rotate:
        cell_label.shapes(layer_label).insert(region, pya.DTrans(pya.DTrans.R90,pya.DVector(position[0], position[1])))
    else:
        cell_label.shapes(layer_label).insert(region, pya.DTrans(pya.DVector(position[0], position[1])))
    return cell_label

In [ ]:
transitions_1x11 = [0.06857, 0.20885, 0.44467, 0.52930, 0.72101, 0.72854, 0.86437]
transitions_1x13 = [0.17765, 0.31352, 0.41244, 0.49846, 0.57633, 0.70857, 0.73041]

In [ ]:

#initialize
layout = pya.Layout()
top = layout.create_cell('top')



# single_calibration = layout.create_cell('rectangle')
# layer_n = layout.layer("layer_n")


layers = np.arange(0,128)




# Create matrix of squares for measurement

period_x = 50
period_y = 50


transitions_x = np.concatenate(([0],np.array(transitions_1x13), [1]))*period_x
transitions_y = np.concatenate(([0],np.array(transitions_1x11), [1]))*period_y

layer_x = 1
layer_y = 2


layer_x = layout.layer("dammannhorizontal")
layer_y = layout.layer("dammannvertical")
cell_dammmann_y = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_y),2):
    start_y = transitions_y[i-1]
    end_y = transitions_y[i]
    print(start_y, end_y)

    cell_dammmann_y.shapes(layer_y).insert(pya.DBox(0, start_y, period_x, end_y))

trans = pya.Trans(pya.Point(0,0))

new_instance = pya.DCellInstArray(cell_dammmann_y.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
top.insert(new_instance)

cell_dammmann_x = layout.create_cell('dammann_horizontal')

for i in range(1,len(transitions_x),2):
    start_x = transitions_x[i-1]
    end_x = transitions_x[i]
    print(start_x, end_x)

    cell_dammmann_x.shapes(layer_x).insert(pya.DBox(start_x, 0, end_x, period_y))

trans = pya.Trans(pya.Point(0,0))

new_instance = pya.DCellInstArray(cell_dammmann_x.cell_index(), trans, pya.Vector(50, 0 ), pya.Vector(0, 50), 1,1)
top.insert(new_instance)




corner_length = 500#*micro
corner_width = 100#*micro

field_width = 48000 #*micro
field_height = 25000 #*micro
field_height = 15000 #*micro

layer = "layer127"
# gscalibration.add_corners(field_width, field_height, corner_length, corner_width, layer)
# label = "text"
# # gscalibration.add_label(label, position=(-4000, -4000), mag=400, layer=layer)


# trans = pya.Trans(pya.Point(0,0))

# top_cell = layout.top_cell()

# cell_corner = create_corners_cell(layout, field_width, field_height, corner_length, corner_width, layer)
# top_cell.insert(pya.DCellInstArray(cell_corner.cell_index(), trans))


# for i in range (16):
#     x0,y0 = -22500, 7250
#     position=(x0+step*i, y0)
#     rotate=False
#     mag=200

#     label = "%d"%(i)
#     cell_label = create_label_cell(layout, label, position, rotate, mag, layer)
#     top_cell.insert(pya.DCellInstArray(cell_label.cell_index(), trans))


layout.write("test.dxf")



# layout.write("gs_calibration_v20240507.dxf")

0.0 3.4285
10.4425 22.2335
26.465 36.0505
36.427 43.2185
0.0 8.8825
15.676000000000002 20.622
24.923000000000002 28.8165
35.4285 36.5205
